In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/car-evaluation/car_evaluation.csv


# Importing Libraries 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Loading Dataset

In [4]:
df = pd.read_csv("/kaggle/input/car-evaluation/car_evaluation.csv", header=None)

since header doesnot contain any valuable information

In [5]:
df.head()

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [6]:
df.shape

(1728, 7)

In [7]:
col_names = ['buying', 'maintinance', 'doors', 'persons', 'lug_boot', 'safety', 'class']
df.columns = col_names

In [8]:
df.head()

,buying,maintinance,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   buying       1728 non-null   object
 1   maintinance  1728 non-null   object
 2   doors        1728 non-null   object
 3   persons      1728 non-null   object
 4   lug_boot     1728 non-null   object
 5   safety       1728 non-null   object
 6   class        1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [10]:
df.dtypes

buying         object
maintinance    object
doors          object
persons        object
lug_boot       object
safety         object
class          object
dtype: object

In [11]:
print(df[col_names].value_counts())

buying  maintinance  doors  persons  lug_boot  safety  class
high    high         2      2        big       high    unacc    1
med     med          4      4        med       med     acc      1
                            more     small     low     unacc    1
                                               high    acc      1
                                     med       med     acc      1
                                                               ..
low     low          3      2        med       med     unacc    1
                                               low     unacc    1
                                               high    unacc    1
                                     big       med     unacc    1
vhigh   vhigh        5more  more     small     med     unacc    1
Name: count, Length: 1728, dtype: int64


In [12]:
df['class'].value_counts()

class
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64

imbalance classes in target variable

In [13]:
df.isnull().sum()

buying         0
maintinance    0
doors          0
persons        0
lug_boot       0
safety         0
class          0
dtype: int64

# Splitting the Dataset into dependent and independent variable 

In [14]:
x = df.drop("class", axis=1)
y = df["class"]

In [15]:
x.head()

,buying,maintinance,doors,persons,lug_boot,safety
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med


In [16]:
y.head()

0    unacc
1    unacc
2    unacc
3    unacc
4    unacc
Name: class, dtype: object

# Spliting Data for Trainig and Testing

In [17]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.20, random_state=42)

In [18]:
x_train.shape, x_test.shape

((1382, 6), (346, 6))

In [19]:
x_train.head()

,buying,maintinance,doors,persons,lug_boot,safety
107,vhigh,vhigh,5more,more,big,high
901,med,vhigh,3,4,small,med
1709,low,low,5more,2,big,high
706,high,med,4,2,med,med
678,high,med,3,2,med,low


# Feature Engineering 

The category_encoders library is a powerful Python library designed to help data scientists and machine learning practitioners convert categorical data into numerical format, which is necessary for most machine learning models.

category_encoders provides a variety of encoding techniques beyond what scikit-learn offers.

| Encoder               | Description                                                                    |
| --------------------- | ------------------------------------------------------------------------------ |
| **OneHotEncoder**     | Converts each category into a separate binary column.                          |
| **OrdinalEncoder**    | Assigns an integer to each category.                                           |
| **BinaryEncoder**     | Converts categories to binary and splits the digits into separate columns.     |
| **TargetEncoder**     | Replaces each category with the average value of the target for that category. |
| **CatBoostEncoder**   | Like target encoding, but handles data leakage better using statistics.        |
| **HelmertEncoder**    | Useful in regression, represents each level compared to mean of later levels.  |
| **SumEncoder**        | Compares each level to the overall mean.                                       |
| **PolynomialEncoder** | Captures interactions and trends by applying polynomial contrast.              |
| **BaseNEncoder**      | Generalization of binary encoding using any base (e.g., base 3, base 4).       |


In [20]:
# from sklearn.preprocessing import category_encoder as ce
import category_encoders as ce

In [21]:
# encode categorical variables with ordinal encoding
encoder = ce.OrdinalEncoder(cols = ['buying', 'maintinance', 'doors', 'persons', 'lug_boot', 'safety'])
x_train = encoder.fit_transform(x_train)
x_test = encoder.transform(x_test)

In [22]:
x_train.head()

,buying,maintinance,doors,persons,lug_boot,safety
107,1,1,1,1,1,1
901,2,1,2,2,2,2
1709,3,2,1,3,1,1
706,4,3,3,3,3,2
678,4,3,2,3,3,3


In [23]:
x_test.head()

,buying,maintinance,doors,persons,lug_boot,safety
599,4,4,3,3,3,1
1201,2,2,4,2,3,2
628,4,4,1,3,1,2
1498,3,4,1,2,3,2
1263,2,2,3,1,3,3


# Random Forest Classifier with Default Parameters 

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
# instantiating the classifier
rfc = RandomForestClassifier(n_estimators=10, random_state=0)
rfc.fit(x_train, y_train)

RandomForestClassifier(n_estimators=10, random_state=0)

In [26]:
y_pred = rfc.predict(x_test)
y_pred

array(['unacc', 'good', 'unacc', 'acc', 'unacc', 'acc', 'unacc', 'unacc',
       'acc', 'unacc', 'acc', 'acc', 'acc', 'unacc', 'unacc', 'unacc',
       'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'acc', 'acc', 'unacc',
       'acc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc',
       'unacc', 'unacc', 'acc', 'good', 'acc', 'unacc', 'unacc', 'unacc',
       'unacc', 'unacc', 'acc', 'acc', 'acc', 'acc', 'unacc', 'unacc',
       'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc',
       'unacc', 'unacc', 'unacc', 'unacc', 'good', 'unacc', 'acc',
       'unacc', 'unacc', 'acc', 'vgood', 'unacc', 'acc', 'acc', 'unacc',
       'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'good', 'unacc',
       'unacc', 'unacc', 'unacc', 'vgood', 'unacc', 'unacc', 'good',
       'vgood', 'acc', 'unacc', 'acc', 'vgood', 'acc', 'unacc', 'good',
       'unacc', 'unacc', 'unacc', 'good', 'unacc', 'unacc', 'unacc',
       'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'good', 'good', 'acc',
      

## Which `average` to choose for f1_score?
The f1_score() function needs an average parameter to handle multiclass targets correctly.

| `average`    | Description                                                               |
| ------------ | ------------------------------------------------------------------------- |
| `'micro'`    | Calculates metrics globally by counting total true positives, etc.        |
| `'macro'`    | Averages the F1 score for each class equally, no matter the support.      |
| `'weighted'` | Like macro, but takes into account class imbalance (recommended usually). |
| `None`       | Returns F1 score for each class individually as an array.                 |


In [27]:
from sklearn.metrics import accuracy_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_pred, y_test, average='weighted')  # or 'macro', 'micro'

print("accuracy:", accuracy)
print("f1-score:", f1)


accuracy: 0.9248554913294798
f1-score: 0.9228525237286206


Model accuracy with 10 decision-trees: 0.92

In [28]:
# instantiating the classifier with n_estimators = 100
rfc100 = RandomForestClassifier(n_estimators=100, random_state=0)
rfc100.fit(x_train, y_train)

# prediction
y_pred100 = rfc100.predict(x_test)

# validating predictions
accuracy100 = accuracy_score(y_test, y_pred100)
f1_100 = f1_score(y_test, y_pred100, average='weighted')  # or 'macro', 'micro'

print("accuracy:", accuracy100)
print("f1-score:", f1_100)



accuracy: 0.9624277456647399
f1-score: 0.9635438386675284


Model accuracy with 100 decision-trees: 0.96

In [29]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print (cm)
print()
cm100 = confusion_matrix(y_test, y_pred100)
print (cm100)

[[ 71   6   6   0]
 [  2   8   0   1]
 [  7   0 228   0]
 [  2   2   0  13]]

[[ 74   6   3   0]
 [  0  10   0   1]
 [  1   0 234   0]
 [  2   0   0  15]]


In [30]:
from sklearn.metrics import classification_report

In [31]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         acc       0.87      0.86      0.86        83
        good       0.50      0.73      0.59        11
       unacc       0.97      0.97      0.97       235
       vgood       0.93      0.76      0.84        17

    accuracy                           0.92       346
   macro avg       0.82      0.83      0.82       346
weighted avg       0.93      0.92      0.93       346



In [32]:
print(classification_report(y_test, y_pred100))

              precision    recall  f1-score   support

         acc       0.96      0.89      0.92        83
        good       0.62      0.91      0.74        11
       unacc       0.99      1.00      0.99       235
       vgood       0.94      0.88      0.91        17

    accuracy                           0.96       346
   macro avg       0.88      0.92      0.89       346
weighted avg       0.97      0.96      0.96       346

